<a href="https://colab.research.google.com/github/Tech-Prep-Mentor/Food-Recommendation-Data/blob/main/ingredients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Food_Recommender/food_dataset/modelling.csv')
data.head()

,name,ingredients_cleaned,days,hours,minutes
0,arriba baked winter squash mexican style,"winter squash, mexican seasoning, mixed spice,...",0,0,55
1,a bit different breakfast pizza,"prepared pizza crust, sausage patty, eggs, mil...",0,0,30
2,all in the kitchen chili,"ground beef, yellow onions, diced tomatoes, to...",0,2,10
3,alouette potatoes,"spreadable cheese with garlic and herbs, new p...",0,0,45
4,amish tomato ketchup for canning,"tomato juice, apple cider vinegar, sugar, salt...",0,3,10


In [ ]:
data.rename(columns={'ingredients_cleaned': 'ingredients'}, inplace=True)
data.rename(columns={'name': 'recipes'}, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   recipes      231637 non-null  object
 1   ingredients  231637 non-null  object
 2   days         231637 non-null  int64 
 3   hours        231637 non-null  int64 
 4   minutes      231637 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 8.8+ MB


In [ ]:
# veggi = ['tomato','parsley','pepper','seed','green','horseradish','zucchini','cabbage','carrot','broccoli','rosemary', 'corn', 'olive', 'celery','fig']
# fruit = ['banana','apple','orange','coconut','grape','grapefruit','melon']
# meat = ['beek','pork','chicken','fish','turkey','bacon','goat','egg','shrimp','crab']
# dairy = ['milk','cheese','yoghurt','butter','cream']
# seasoning = ['salt','sugar','peper','chilli','oil','sesame','sauce']np

In [ ]:
# Standard normalize time values
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
standard = scaler.fit_transform(data[['days','hours','minutes']])
data[['days','hours','minutes']] = standard
data

,recipes,ingredients,days,hours,minutes
0,arriba baked winter squash mexican style,"winter squash, mexican seasoning, mixed spice,...",-0.021556,-0.408463,2.011559
1,a bit different breakfast pizza,"prepared pizza crust, sausage patty, eggs, mil...",-0.021556,-0.408463,0.394007
2,all in the kitchen chili,"ground beef, yellow onions, diced tomatoes, to...",-0.021556,0.809477,-0.900034
3,alouette potatoes,"spreadable cheese with garlic and herbs, new p...",-0.021556,-0.408463,1.364538
4,amish tomato ketchup for canning,"tomato juice, apple cider vinegar, sugar, salt...",-0.021556,1.418447,-0.900034
...,...,...,...,...,...
231632,zydeco soup,"celery, onion, green sweet pepper, garlic clov...",-0.021556,0.200507,-1.547055
231633,zydeco spice mix,"paprika, salt, garlic powder, onion powder, dr...",-0.021556,-0.408463,-1.223544
231634,zydeco ya ya deviled eggs,"hard-cooked eggs, mayonnaise, dijon mustard, s...",-0.021556,-0.408463,1.041028
231635,cookies by design cookies on a stick,"butter, eagle brand condensed milk, light brow...",-0.021556,-0.408463,0.329305


In [ ]:
import spacy

In [ ]:
%%python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load('en_core_web_md')

# embed
embeddings = np.array([nlp(item).vector for item in data['ingredients']])

KeyboardInterrupt: 

In [ ]:
embeddings.shape

NameError: name 'embeddings' is not defined

In [ ]:
# Save embeddings as df
embed = pd.DataFrame(embeddings)
# embed.to_csv('..\data\string_embed.csv', index=False)
embed

In [ ]:
# ingre1 = nlp('pork')
# ingre2 = nlp('beef')
# ingre1.similarity(ingre2)

In [ ]:
df = data.join(embed)
df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

preference = "cheese, chicken, tomato, pepper"
preference = nlp(preference).vector
time = [2, 23, 50] # 2 days, 23 hours, 50 minutes

# stack together preference and time to create request
request = np.hstack([time, preference])
# request = preference
request = request.reshape(1, -1)
request = pd.DataFrame(request)
request

In [ ]:
# cosine similarity
cosine_sim = cosine_similarity(request, df.drop(columns=['recipes','ingredients','days','hours','minutes']))

n_rec = 10
rec_indices = np.argsort(cosine_sim)[0][::-1]

recommendations = data.loc[rec_indices][:n_rec]
recommendations

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModel

In [ ]:
# # Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

In [ ]:
# inputs = tokenizer(data['ingredients'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=10)

In [ ]:
# # Get the embeddings
# with torch.no_grad():
#     outputs = model(**inputs)

# # `outputs` is a dictionary with several components. `last_hidden_state` contains the embeddings
# embeddings = outputs.last_hidden_state

In [ ]:
# # We can take the mean of the token embeddings to get a single vector per ingredient
# ingredient_embeddings = embeddings.mean(dim=1)

# # (shape n_ingredients x 768)
# ingredient_embeddings

In [ ]:
# import nltk
# nltk.download('punkt')
# # from nltk.cluster import KMeansClusterer
# from nltk.tokenize import word_tokenize
# # from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import StandardScaler

In [ ]:
# # Tokenize the ingredients
# ingredient_tokens = [word_tokenize(ingredient) for ingredient in data['ingredients']]

# # Convert tokens back to string for TF-IDF vectorization
# ingredient_texts = [' '.join(tokens) for tokens in ingredient_tokens]

# data_set = set(data)
# ingre_set = set(ingredient_texts)

# # Find the difference
# missing_in_tokenized = data_set - ingre_set
# missing_in_tokenized

In [ ]:
# # Find difference between data and ingredient_text
# data['ingredients'] = ingredient_texts

In [ ]:
# # TF-IDF vectorization
# vectorizer = TfidfVectorizer(stop_words='english')
# X = vectorizer.fit_transform(ingredient_texts)

# # Normalize the TF-IDF matrix
# scaler = StandardScaler()
# X_normalized = scaler.fit_transform(X.toarray())

In [ ]:
# # Apply KMeans clustering with NLTK
# num_clusters = 5  # Adjust as needed
# kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
# assigned_clusters = kclusterer.cluster(X_normalized, assign_clusters=True)

# # Visualize the clusters
# plt.figure(figsize=(10, 6))
# plt.scatter(X_normalized[:, 0], X_normalized[:, 1], c=assigned_clusters, cmap='viridis')
# plt.xlabel('Normalized TF-IDF Feature 1')
# plt.ylabel('Normalized TF-IDF Feature 2')
# plt.title('Clusters of Ingredients')
# plt.show()

In [ ]:
# import numpy as np

# # Get cluster means
# cluster_means = np.array(kclusterer.means())

# # Calculate distances between each point and cluster means
# distances = np.linalg.norm(X_normalized[:, np.newaxis] - cluster_means, axis=2)

# # Assign labels based on the nearest cluster mean
# labels = np.argmin(distances, axis=1)

# data['cluster'] = labels
# data.head()